# Inferenta cu model siamez

In [38]:
import torch
from torchvision import transforms
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import argparse
import random
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy as np

In [25]:
class FingerprintDatasetDirect(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []

        # Collecting all images in the dataset
        for class_dir in os.listdir(data_dir):
            class_path = os.path.join(data_dir, class_dir)
            if os.path.isdir(class_path):
                for image_name in os.listdir(class_path):
                    self.image_paths.append(os.path.join(class_path, image_name))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the first image
        img1_path = self.image_paths[idx]
        img1 = Image.open(img1_path).convert('RGB')

        # Randomly decide if we want a positive or negative pair
        is_positive = random.choice([True, False])

        if is_positive:
            # Positive pair: same image loaded twice
            img2_path = img1_path
            img2 = Image.open(img2_path).convert('RGB')
        else:
            # Negative pair: randomly select a different image
            img2_path = random.choice(self.image_paths)
            while img2_path == img1_path:  # Ensure it's a different image
                img2_path = random.choice(self.image_paths)
            img2 = Image.open(img2_path).convert('RGB')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, int(is_positive)  # Return 1 for positive, 0 for negative


In [26]:
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=256):
        super(SiameseNetwork, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7),  # Output: [64, 99, 99]
            nn.ReLU(),
            nn.MaxPool2d(2),                 # -> [64, 49, 49]

            nn.Conv2d(64, 128, kernel_size=5),  # -> [128, 45, 45]
            nn.ReLU(),
            nn.MaxPool2d(2),                   # -> [128, 22, 22]

            nn.Conv2d(128, 128, kernel_size=3),  # -> [128, 20, 20]
            nn.ReLU(),
            nn.MaxPool2d(2)                     # -> [128, 10, 10]
        )

        self.fc = nn.Sequential(
            nn.Linear(128 * 10 * 10, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim)  # Final output is the embedding
        )

    def forward_one(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

In [45]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((105, 105)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [44]:
def get_top_matches(test_image_path, model, dataset, transform, top_k=5, device='cpu'):
    test_image = Image.open(test_image_path).convert('RGB')
    test_image = transform(test_image).unsqueeze(0).to(device)

    model.eval()

    similarity_scores = []

    for img_path in tqdm(dataset.image_paths, desc="Matching", unit="img"):
        if os.path.isfile(img_path):
            db_image = Image.open(img_path).convert('RGB')
            db_image = transform(db_image).unsqueeze(0).to(device)

            with torch.no_grad():
                output_db, output_test = model(db_image, test_image)
                output_db = F.normalize(output_db, p=2, dim=1)
                output_test = F.normalize(output_test, p=2, dim=1)

                similarity_score = torch.cosine_similarity(output_test, output_db).item()
                similarity_scores.append((img_path, similarity_score))

    # Sort and get top K
    top_matches = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_k]



    return top_matches


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
model_file = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/Models/Fingerprints/siamese_model_min_loss_32.pth'

database_dir = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_fragment'

embedding_size = 32
dataset = FingerprintDatasetDirect(database_dir, transform)


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork(embedding_dim=embedding_size).to(device)
model.load_state_dict(torch.load(model_file, weights_only=True, map_location=torch.device(device)))

<All keys matched successfully>

In [42]:
uploaded1 = files.upload()

for fn in uploaded1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded1[fn])))

first_image = fn

Saving class1_Arc_0032.png to class1_Arc_0032 (2).png
User uploaded file "class1_Arc_0032 (2).png" with length 187849 bytes


In [49]:
top_matches = get_top_matches(first_image, model, dataset, transform, device=device)

Matching: 100%|██████████| 261/261 [00:18<00:00, 14.13img/s]


In [50]:
top_k = 5

print(f"\nTop {top_k} matches:")
for i, (path, score) in enumerate(top_matches, 1):
    print(f"{i}. {path} — Score: {score:.4f}")


Top 5 matches:
1. /content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_fragment/class1_Arc/class1_Arc_0032.png — Score: 1.0000
2. /content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_fragment/class3_Loop/class3_Loop_0032.png — Score: 0.9818
3. /content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_fragment/class3_Loop/class3_Loop_0049.png — Score: 0.9813
4. /content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea informatiei/Curs/ColabMount/DATA/NISTDB4_fragment/class3_Loop/class3_Loop_0117.png — Score: 0.9676
5. /content/drive/MyDrive/Facultate Informatica/Profesor/2024-2025/Sisteme expert si metode biometrice in securitatea 